In [1]:
import numpy as np
from numpy import dot
from filterpy.common import pretty_str, reshape_z
import scipy.linalg as linalg
from filterpy.stats import logpdf

# Kalman Filter Algorithm

**Initialization**

    1. Initialize the state of the filter
    2. Initialize our belief in the state
    
**Predict**

    1. Use process model to predict state at the next time step
    2. Adjust belief to account for the uncertainty in prediction    
**Update**

    1. Get a measurement and associated belief about its accuracy
    2. Compute residual between estimated state and measurement
    3. Compute scaling factor based on whether the measurement
    or prediction is more accurate
    4. set state between the prediction and measurement based 
    on scaling factor
    5. update belief in the

# Equation Reference

$$
\begin{aligned}
\text{Predict Step}\\
\mathbf{\bar x} &= \mathbf{F x} + \mathbf{B u} = PriorState\\ 
\mathbf{\bar P} &= \mathbf{FP{F}}^\mathsf T + \mathbf Q = ProcessNoise\\
\\
\text{Update Step}\\
\textbf{S} &= \mathbf{H\bar PH}^\mathsf T + \mathbf R = ObtainSystemUncertainty\\
\mathbf K &= \mathbf{\bar PH}^\mathsf T \mathbf{S}^{-1} = KalmanGain\\
\textbf{y} &= \mathbf z - \mathbf{H \bar x} = Residual\\
\mathbf x &=\mathbf{\bar x} +\mathbf{K\textbf{y}} = UpdatedState(Posterior)\\
\mathbf P &= (\mathbf{I}-\mathbf{KH})\mathbf{\bar P = UpdateNoise}
\end{aligned}
$$

#### Initial State (x)

$$\mathbf x 
= \begin{bmatrix}position (m)\\ velocity (m/s)\end{bmatrix} 
= \begin{bmatrix}x\\ v\end{bmatrix} 
= \begin{bmatrix}0\\ 0\end{bmatrix}$$

#### State transition function (F)

$$\mathbf F = 
\begin{bmatrix}1&\Delta t  \\ 0&1\end{bmatrix}$$

#### Control Variable (u)

$$\mathbf u = 
\begin{bmatrix}a\end{bmatrix}$$

#### Control Matrix (B)

$$\mathbf B = 
\begin{bmatrix}1/2\Delta t^2 \\ \Delta t\end{bmatrix}$$

#### Covariance Matrix (P)

Position $\mathbf x$ variance ($\sigma_\mathtt{x}^2$) selected at 5m 

Velocity $\mathbf v$ variance ($\sigma_\mathtt{v}^2$) selected at 5m/s

Covariance $\sigma_\mathtt{xv} = \sigma_\mathtt{x}\sigma_\mathtt{v} $

$$\mathbf P 
= \begin{bmatrix}\sigma_\mathtt{x}^2&\sigma_x\sigma_v  \\ \sigma_v\sigma_x&\sigma_\mathtt{v}^2\end{bmatrix} 
= \begin{bmatrix}5&5  \\ 5&5\end{bmatrix}$$

#### Process Noise (Q)

$$\mathbf Q 
= \begin{bmatrix}0&0 \\ 0&1\end{bmatrix}
$$

#### Measurement Noise (R)

Need to double check sensors

$$\mathbf x 
= \begin{bmatrix}100\end{bmatrix}
$$

### State/Measurement Related Initialization

In [2]:
state                = np.array([[0,0]]).T                  # state

time_step            = 1                               # time step

state_transition     = np.array([[1, time_step],       # state transition matrix
                                 [0, 1]])

control_input        = np.array([[10]])                  # control input 

control_matrix       = np.array([[0.5*(time_step**2)], # control matrix
                                 [time_step]])

measurement_data     = np.array([[10]])                  # measurement
  
measurement_function = np.array([[1,0]])                 # measurement function
                                

### Noise Related Initialization

In [3]:
position_variance = 5
velocity_variance = 5
position_std      = np.sqrt(position_variance)
velocity_std      = np.sqrt(velocity_variance)
covariance_value  = position_std * velocity_std

covariance_matrix = np.array([[position_variance, covariance_value],
                              [covariance_value,  position_variance]])

process_noise     = np.array([[0,0],
                              [0,1]]) 

measurement_noise = np.array([[4]])

# Math in terms of Matrices

$$
\begin{aligned}
\text{Prediction Step}\\
\mathbf{\bar x} &= \mathbf{F x} + \mathbf{B u} = PriorState\\ 
\mathbf{\bar P} &= \mathbf{FP{F}}^\mathsf T + \mathbf Q = ProcessNoise\\
\end{aligned}
$$


$$
\begin{aligned}
\mathbf{\bar x} &= \mathbf{F x} + \mathbf{B u}\\ 
\mathbf{\bar x} &= \begin{bmatrix}1&\Delta t  \\ 0&1\end{bmatrix} \begin{bmatrix}x  \\ v\end{bmatrix} 
+ \begin{bmatrix}1/2\Delta t^2 \\ \Delta t\end{bmatrix} 
\begin{bmatrix}a\end{bmatrix} \\
\mathbf{\bar x} &= \begin{bmatrix}1&1  \\ 0&1\end{bmatrix} \begin{bmatrix}0  \\ 0\end{bmatrix} + 
\begin{bmatrix}0.5*1 \\ 1\end{bmatrix} 
\begin{bmatrix}10\end{bmatrix} \\
\mathbf{\bar x} &= \begin{bmatrix}5 \\ 10\end{bmatrix} 
\end{aligned}
$$

$$
\begin{aligned}
\mathbf{\bar P} &= \mathbf{FP{F}}^\mathsf T + \mathbf Q = ProcessNoise\\
\mathbf{\bar x} &= \begin{bmatrix}1&\Delta t  \\ 0&1\end{bmatrix} \begin{bmatrix}\sigma_\mathtt{x}^2&\sigma_x\sigma_v \\ \sigma_v\sigma_x&\sigma_\mathtt{v}^2\end{bmatrix} \begin{bmatrix}1&0  \\ \Delta t&1\end{bmatrix}
+
\begin{bmatrix}0&0  \\ 0&1\end{bmatrix}
\end{aligned}
$$

In [4]:
def predict(x, P, F=1, Q=0, u=0, B=1, alpha=1.):
    if np.isscalar(F): 
        F = np.array(F)
    x = np.dot(F, x) + np.dot(B, u) # Fx + Bu
    P = (alpha * alpha) * np.dot(np.dot(F, P), F.T) + Q # P=(1^2) 

    return x, P

x_pred, P_pred = predict(x=state, P=covariance_matrix, F=state_transition, Q=process_noise, u=control_input, B=control_matrix)

print("State is")
print(x_pred)
print("\nCovariance is")
print(P_pred)

State is
[[ 5.]
 [10.]]

Covariance is
[[20. 10.]
 [10.  6.]]


$$
\begin{aligned}
\text{Update Step}\\
\textbf{S} &= \mathbf{H\bar PH}^\mathsf T + \mathbf R = ObtainSystemUncertainty\\
\mathbf K &= \mathbf{\bar PH}^\mathsf T \mathbf{S}^{-1} = KalmanGain\\
\textbf{y} &= \mathbf z - \mathbf{H \bar x} = Residual\\
\mathbf x &=\mathbf{\bar x} +\mathbf{K\textbf{y}} = UpdatedState(Posterior)\\
\mathbf P &= (\mathbf{I}-\mathbf{KH})\mathbf{\bar P = UpdateNoise}
\end{aligned}
$$

In [5]:
def update(x, P, z, R, H=None, return_all=False):
    """
    Add a new measurement (z) to the Kalman filter. If z is None, nothing
    is changed.

    This can handle either the multidimensional or unidimensional case. If
    all parameters are floats instead of arrays the filter will still work,
    and return floats for x, P as the result.

    update(1, 2, 1, 1, 1)  # univariate
    update(x, P, 1



    Parameters
    ----------

    x : numpy.array(dim_x, 1), or float
        State estimate vector

    P : numpy.array(dim_x, dim_x), or float
        Covariance matrix

    z : (dim_z, 1): array_like
        measurement for this update. z can be a scalar if dim_z is 1,
        otherwise it must be convertible to a column vector.

    R : numpy.array(dim_z, dim_z), or float
        Measurement noise matrix

    H : numpy.array(dim_x, dim_x), or float, optional
        Measurement function. If not provided, a value of 1 is assumed.

    return_all : bool, default False
        If true, y, K, S, and log_likelihood are returned, otherwise
        only x and P are returned.

    Returns
    -------

    x : numpy.array
        Posterior state estimate vector

    P : numpy.array
        Posterior covariance matrix

    y : numpy.array or scalar
        Residua. Difference between measurement and state in measurement space

    K : numpy.array
        Kalman gain

    S : numpy.array
        System uncertainty in measurement space

    log_likelihood : float
        log likelihood of the measurement
    """

    #pylint: disable=bare-except

    if z is None:
        if return_all:
            return x, P, None, None, None, None
        return x, P

    if H is None:
        H = np.array([1])

    if np.isscalar(H):
        H = np.array([H])

    Hx = np.atleast_1d(dot(H, x))
    z = reshape_z(z, Hx.shape[0], x.ndim)

    # error (residual) between measurement and prediction
    y = z - Hx

    # project system uncertainty into measurement space
    S = dot(dot(H, P), H.T) + R


    # map system uncertainty into kalman gain
    try:
        K = dot(dot(P, H.T), linalg.inv(S))
    except:
        # can't invert a 1D array, annoyingly
        K = dot(dot(P, H.T), 1./S)


    # predict new x with residual scaled by the kalman gain
    x = x + dot(K, y)

    # P = (I-KH)P(I-KH)' + KRK'
    KH = dot(K, H)
    print('\nKH')
    print(KH)
    print('\nKH shape')
    print(np.eye(KH.shape[0]))
        
    try:
        I_KH = np.eye(KH.shape[0]) - KH
        #print('I_KH')
        #print(I_KH)
        #I_KH = I - dot(K, H)
        print('\nI_KH')
        print(I_KH)
    except:
        I_KH = np.array([1 - KH])
        print('\nI_KH')
        print(I_KH)
    # I_KH = np.array([1 - KH])
    P = dot(dot(I_KH, P), I_KH.T) + dot(dot(K, R), K.T)


    if return_all:
        # compute log likelihood
        log_likelihood = logpdf(z, dot(H, x), S)
        return x, P, y, K, S, log_likelihood
    return x, P

x, P, y, K, S, log_likelihood = update(x=x_pred, P=P_pred, z=measurement_data, R=measurement_noise, H=measurement_function, return_all=True)

print('\nMeasurement Uncertainty')
print(S)
print('\nKalman Gain')
print(K)
print('\nResidual')
print(y)
print('\nEstimated State')
print(x)
print('\nEstimated Covariance')
print(P)


KH
[[0.83333333 0.        ]
 [0.41666667 0.        ]]

KH shape
[[1. 0.]
 [0. 1.]]

I_KH
[[ 0.16666667  0.        ]
 [-0.41666667  1.        ]]

Measurement Uncertainty
[[24.]]

Kalman Gain
[[0.83333333]
 [0.41666667]]

Residual
[[5.]]

Estimated State
[[ 9.16666667]
 [12.08333333]]

Estimated Covariance
[[3.33333333 1.66666667]
 [1.66666667 1.83333333]]


In [6]:
test1 = np.array([[0.83,0.41]]).T
test2 = np.array([[1,0]])
test3 = np.dot(test1,test2)

print(test3)

[[0.83 0.  ]
 [0.41 0.  ]]


In [7]:
test1 = np.array([[0.17,1],
                  [0.59,1]])
test2 = np.array([[20, 10], 
                  [10, 6]])
test3 = np.dot(test1,test2)

print(test3)

[[13.4  7.7]
 [21.8 11.9]]
